### Objectives: 
    1. Import AquaStat data found on figshare
    2. Map data with folium to determine geographic subset
    3. Subset data to Southwest or Rockies?  Looking for lakes that might overlap with
    MTBS

Used with earth-analytics environment

## 1. Import packages / set working directory

In [1]:
import earthpy as et
import os 
import io
import requests
import feather
import datatools
import folium
import urllib
import json
import ee
import geemap
import numpy as np
#from pandas.io.json import json_normalize
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame
from shapely.geometry import Point
import warnings
#from shapely.geometry import Point

# Initialize Google Earth Engine to grab landsat images in section 4
#ee.Authenticate()
ee.Initialize()

ModuleNotFoundError: No module named 'datatools'

In [ ]:
# Set working directory

# if the desired path exists:
data_dir = os.path.join(et.io.HOME, 'Dropbox', 'cu_earthdata_certificate_2021', 'earthlab_project', 'data')
if os.path.exists(data_dir):
    # set working directory:
    os.chdir(data_dir)
    print("path exists")
else:
    print("path does not exist, making new path")
    os.makedirs(data_dir)
    os.chdir(data_dir)

## 2. Download the region shapefile 
to use in clipping the MTBS and HydroLAKES dataset

In [ ]:
lis = ['Colorado','Utah', 'Wyoming', 'New Mexico', 'Arizona']
states = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.inList('NAME', lis))

states_gdf = geemap.ee_to_geopandas(states, selectors = ['NAME'])
print("The crs of your area of interest df is", states_gdf.crs)
states_gdf.head()

bounding_box = states_gdf.envelope
states_bb = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
states_bb

sw_union = states_bb.dissolve() # dissolve states to one poly for maps below

In [ ]:
# Subset this dataframe one more time, extracting only data that align with burned lake dataset created
# in script 1
# This is the dataset that includes the whole 2km 'catchment' of lakes that overlap with mtbs perimeters.
# Any data inside of these catchements should be evaluated, not just within the actual overlapping polygon 
# (e.g. landsat_n_burnpolys.shp) 

mtbslakes_fp = os.path.join("whole_burned_lakes.shp")
mtbslakes = gpd.read_file(mtbslakes_fp).to_crs(epsg=4326)

# Bring in burn perimeters 
mtbs_fp = os.path.join( "mtbs_polys", "mtbs_polys.shp")
fire_perims = gpd.read_file(mtbs_fp).to_crs(epsg=4326)

## 4. Read AquaSat dataset 
   subset to burned lakes from mtbslakes dataset

In [ ]:
# Read in aquaSat .csv and subset to region of interest

#https://figshare.com/
aquasat_url = 'https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/18733733/sr_wq_rs_join.csv'

# Define output file
file_path = os.path.join("aquasat.csv")

if not os.path.isfile(file_path):    
    aquasat = pd.read_csv(aquasat_url, index_col=0)
    aquasat.to_csv(output_fp)
    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(aquasat['long'], aquasat['lat'])]
    # Creating a Geographic data frame 
    gdf = gpd.GeoDataFrame(aquasat, crs='epsg:4326', geometry=geometry)
    
else:
    aquasat = pd.read_csv(file_path)
    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(aquasat['long'], aquasat['lat'])]
    # Creating a Geographic data frame 
    gdf = gpd.GeoDataFrame(aquasat, crs='epsg:4326', geometry=geometry)
   

In [ ]:
# merge aquasat with fire affected lake dataset to retain aquasat points
aquasat_burn_points = gpd.overlay(mtbslakes, gdf, how ='intersection', keep_geom_type=False)


# merge aquasat with fire affected lake dataset to retain lake catchment polys
aquasat_burn_polys = gpd.overlay(mtbslakes, gdf, how ='union', keep_geom_type=True)

In [ ]:
aquasat_burn_polys.head() 

Now aquasat_burn is a dataset of burned lakes with chla and some landsat/color data. It does not yet have LimnoSat's dWL data. 

Export this as a shapefile to use when collecting landscape data in the next notebooks:

In [ ]:
out_path = os.path.join("burned_w_aquasat_points.shp")
aquasat_burn_points.to_file(out_path)


out_path = os.path.join("burned_w_aquasat_polys.shp")
aquasat_burn_polys.to_file(out_path)
aquasat_burn_polys.to_file("burned_w_aquasat_polys.geojson", driver='GeoJSON')

In [ ]:
# Let's see how the data is disbursed
# plot subsets
fig, ax = plt.subplots(figsize=(16, 12))

sw_union['geometry'].plot(ax=ax, 
                color = 'brown',
                 alpha = 0.5);

mtbslakes['geometry'].plot(ax=ax, 
                color='blue', 
                alpha=1);


aquasat_burn_points['geometry'].plot(ax=ax, 
                color = 'black',
                 alpha = 0.5);

ax.set(facecolor = "grey")
#.set_axis_off()

In [2]:
# Stamen Terrain
map = folium.Map(location = [37,-105], tiles = "Stamen Terrain", zoom_start = 5)
map

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in aquasat_burn_points.geometry ]

i = 0
for coordinates in geo_df_list:
    map.add_child(folium.Marker(location = coordinates,
                            popup =
                            "Hylak_id: " + str(aquasat_burn.Hylak_id[i]) + '<br>' +
                            "ChlA: " + str(aquasat_burn.chl_a[i]) + '<br>' +
                            "Coordinates: " + str(geo_df_list[i]),
                            icon = folium.Icon(color = 'green')))
    i = i + 1

for _, f in fire_perims.iterrows():
    #sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(f['geometry']).simplify(tolerance=0.001)
    geo_f = sim_geo.to_json()
    geo_f = folium.GeoJson(data=geo_f,
                           style_function=lambda x: {'fillColor': 'orange', 'color': 'orange'})
    geo_f.add_child(folium.Popup(f['Incid_Name']))
    #folium.Popup(r['Incid_Name']).add_to(geo_j)
    geo_f.add_to(map)
    
for _, r in mtbslakes.iterrows():
    #without simplifying the representation of each borough, the map might not be displayed
    #sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'blue'})
    #folium.Popup(r['Hylak_id']).add_to(geo_j)
    geo_j.add_child(folium.Popup(r['Hylak_id']))
    geo_j.add_to(map)

map

NameError: name 'folium' is not defined

In [ ]:
# Bring in mtbs data (whole_burned_lakes is HydroLakes and LimnoSat data)
in_path = os.path.join("landsat_n_burnpolys.shp")
landsat_n_burnpolys = gpd.read_file(in_path, bbox=sw_union).to_crs('epsg:4326')

In [ ]:
# Make sure Hylak_id is an integer to match aquasat_burn
# merge aquasat with fire affected lake dataset
aquasat_burn_landsat = gpd.overlay(aquasat_burn_points, landsat_n_burnpolys,  how ='union', keep_geom_type= 'TRUE')

Both aquasat_burn and landsat_n_burnpolys contain the Hylak_id unique identifier.
Merge these datasets based on Hylak_id (subsetting landsat-n-burnpolys to those with 
insitu data)

In [ ]:
# Let's see how the data is disbursed
# plot subsets
fig, ax = plt.subplots(figsize=(16, 12))

sw_union['geometry'].plot(ax=ax, 
                color = 'palegoldenrod',
                 alpha = 0.5);

aquasat_burn['geometry'].plot(ax=ax, 
                color = 'black',
                 alpha = 1);

# landsat_n_burnpolys['geometry'].plot(ax=ax, 
#                 color='white', 
#                 alpha=.25);


# aquasat_burn_landsat['geometry'].plot(ax=ax, 
#                 color='pink', 
#                 alpha=.50);

ax.set(facecolor = "grey")
#.set_axis_off()

In [ ]:
# Generate pre or post fire classification of Landsat imagery based on condition
# that image date is before ingition date

# Convert objects to dates
#aquasat_burn_landsat['date_unity'] = aquasat_burn_landsat['date_unity'].strftime("%B %d, %Y")
aquasat_burn_landsat['date_unity'] = pd.to_datetime(aquasat_burn_landsat['date_unity']).dt.tz_localize(None)
aquasat_burn_landsat['Ig_Date'] = pd.to_datetime(aquasat_burn_landsat['Ig_Date'])
aquasat_burn_landsat.date_unity
#aquasat_burn_landsat.Ig_Date

In [ ]:
aquasat_burn_landsat['pre_post'] = np.where(
    aquasat_burn_landsat['date_unity'] < aquasat_burn_landsat['Ig_Date'], "pre-fire", "post-fire")
# print(colo_tabular.pre_post)

# Generate a calculation for the lapse of time pre- or post fire for plotting
aquasat_burn_landsat['days_since'] = (aquasat_burn_landsat['date_unity'] - aquasat_burn_landsat['Ig_Date'])

# Convert days_since to numerical/integer value that can be plotted
aquasat_burn_landsat.days_since = aquasat_burn_landsat.days_since.astype(
    'timedelta64[D]').astype(int)
# print(aquasat_burn_landsatinsitu_burned_lakes.days_since)

# Add a image month column for trend monitoring
aquasat_burn_landsat['img_month'] = pd.DatetimeIndex(aquasat_burn_landsat['date_unity']).month

# Add a classification that identifies the number of years since the fire.
# This method allows for manual adjustment of year assingment
col = aquasat_burn_landsat['days_since']
conditions = [(col < 2920) & (col >= 2555),
              (col < 2555) & (col >= 2190),
              (col < 2190) & (col >= 1825),
              (col < 1825) & (col >= 1460),
              (col < 1460) & (col >= 1095),
              (col < 1095) & (col >= 730),
              (col < 730) & (col >= 365),
              (col >= 0) & (col < 365),
              (col < 0) & (col >= -365),
              (col < -365) & (col >= -730),
              (col < -703) & (col >= -1095),
              (col < -1095) & (col >= -1460),
              (col < -1460) & (col >= -1825),
              (col < -1825) | (col >= -2190),
              (col < -2190) | (col >= -2555),
              (col < -2555) | (col >= -2920)]
choices = ['8', '7', '6', '5', '4', '3', '2', '1',
           '0', '-1', '-2', '-3', '-4', '-5', '-6', '-7']
aquasat_burn_landsat['years_since'] = np.select(conditions, choices, default='meh')
aquasat_burn_landsat['years_since'] = aquasat_burn_landsat['years_since'].astype(int)

In [ ]:
col = aquasat_burn_landsatinsitu_burned_lakes['dWL']
conditions = [(col == 583) & (col > 581),
              (col <= 581) & (col > 579),
              (col <= 579) & (col > 577),
              (col <= 577) & (col > 575),
              (col <= 575) & (col > 573),
              (col <= 573) & (col > 571),
              (col <= 571) & (col > 570),
              (col <= 570) & (col > 569),
              (col <= 569) & (col > 568),
              (col <= 568) & (col > 567),
              (col <= 567) & (col > 564),
              (col <= 564) & (col > 559),
              (col <= 559) & (col > 549),
              (col <= 549) & (col > 530),
              (col <= 530) & (col > 509),
              (col <= 509) & (col > 495),
              (col <= 495) & (col > 489),
              (col <= 489) & (col > 485),
              (col <= 485) & (col > 480),
              (col <= 480) & (col > 475),
              (col > 470) & (col <= 475)]


choices = ['21', '20', '19', '18', '17', '16', '15', '14', '13', '12', '11',
           '10', '9', '8', '7', '6', '5', '4', '3', '2', '1']
aquasat_burn_landsat['fui'] = np.select(conditions, choices, default='1001')
aquasat_burn_landsat['fui'] = pd.to_numeric(aquasat_burn_landsat['fui'], errors='coerce')

choices = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
           'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u']
aquasat_burn_landsat['fui_color'] = np.select(conditions, choices, default='v')

# Check that new or converted values make sense
aquasat_burn_landsatinsitu_burned_lakes[['Ig_Date', 'pre_post',
              'days_since', 'img_month', 'years_since', 'dWL', 'fui']].head

# colo_tabular.dtypes

In [ ]:
aquasat_burn_landsat.Incid_Name.unique()

In [ ]:
# Export 
# Export file to local drive
out_path = os.path.join("burned_lakes_color_and_insitu.csv")
aquasat_burn_landsat.to_csv(out_path)


## 5. Plots
Looking at pre and post fire data

In [ ]:
plot_subset = iaquasat_burn_landsat.loc[(aquasat_burn_landsat['fui'] != 1001)]

# Plot 1
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 12))

ax1.scatter(x=plot_subset['days_since'],
            y=plot_subset['chl_a'],
            color='green')
ax1.set(title='chl_a in response to time since fire',
       xlabel='Days since fire ignition',
       ylabel='chl_a')
ax1.set_yscale('log')



# Plot 2
ax2.scatter(x=plot_subset['days_since'],
            y=plot_subset['tss'],
            color='brown')
ax2.set(title='TSS in response to time since fire',
       xlabel='Days since fire ignition',
       ylabel='Total suspended solids')

# Plot 3
ax3.scatter(x=plot_subset['days_since'],
            y=plot_subset['doc'],
            color='purple')
ax3.set(title='DOC in response to time since fire',
       xlabel='Days since fire ignition',
       ylabel='DOC')


# Plot 4
ax4.scatter(x=plot_subset['days_since'],
            y=plot_subset['secchi'],
            color='gray')
ax4.set(title='Secchi depth in response to time since fire',
       xlabel='Days since fire ignition',
       ylabel='Secchi depth')

plt.tight_layout()